In [57]:
pip install pandera

In [58]:
pip install gcsfs

In [59]:
import os
import pandas as pd
from google.cloud import storage

In [60]:
serviceAccount = '/content/Chave Ingestao Apache.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#leitura do arquivo em JSON
df = pd.read_json(r'gs://projeto-final-grupo09/entrada_dados/Projeto Final', lines = True)
df.head(1)

In [63]:
filtro = df.property_type != 'PH'
df2 = df[filtro]

In [ ]:
#Renomeando colunas
df2.rename(columns={'id':'identificacao','created_on':'criado_em','operation':'operacao','property_type':'tipo_propriedade',
                   'place_name':'nome_do_local','place_with_parent_names':'pais_local','country_name':'pais','state_name':'estado',
                   'geonames_id':'g_nomes','lat_lon':'latitude_longitude','lat':'latitude','lon':'longitude','price':
                   'preco_cheio','currency':'moeda','price_aprox_local_currency':'preco',
                   'price_aprox_usd':'preco_aproximado_dolar','surface_total_in_m2':'area_total_por_m2',
                   'surface_covered_in_m2':'area_construcao_em_m2','price_usd_per_m2':'preco_dolar_por_m2',
                   'price_per_m2':'preco_por_m2','floor':'andar','rooms':'quartos','expenses':'despesas',
                   'properati_url':'url_da_propriedade','description':'descricao', 'title':'titulo',
                   'image_thumbnail':'miniatura_imagem'}, inplace = True)
df2.head(2)


In [65]:
#chamar a coluna de operacao para ver se tem algo além de venda (no caso a coluna só tem sell - venda - entao irei dropar posteriormente)
sorted(pd.unique(df2['operacao']))

['sell']

In [66]:
#chamar a coluna de país para ver se tem algo além de Brasil (no caso só tem Brasil - então iremos dropar posteriormente)
sorted(pd.unique(df2['pais']))

['Brasil']

In [67]:
#chamar a coluna de moeda para ver se tem algo além de BRL (no caso só tem BRL - então iremos dropar posteriormente)
sorted(pd.unique(df2['moeda']))

['', 'BRL', 'COP']

In [ ]:
#criacao de variavel - colunas - para posterior drop
colunas = ['operacao', 'pais', 'moeda', 'latitude_longitude', 'latitude', 'longitude', 'preco_aproximado_dolar', 'pais_local', 
'preco_dolar_por_m2', 'despesas', 'descricao', 'titulo', 'miniatura_imagem', 'url_da_propriedade', 'andar', 'preco_cheio']
df2.drop(colunas, axis=1, inplace=True)

In [ ]:
#verificando se há (e quantos são) os valores na coluna nome_do_local
df2['nome_do_local'].value_counts()

In [ ]:
#traduzindo as informações contidas na coluna de tipo_propriedade
df2['tipo_propriedade'].replace(['house', 'apartment', 'store'],['casa','apartamento','loja'], inplace = True)

In [71]:
#verificar se há apenas um valor na coluna (no caso a coluna propriedade_tipo tem 3 informações significativas e uma (PH) que será dropada)
sorted(pd.unique(df2['tipo_propriedade']))

['apartamento', 'casa', 'loja']

In [72]:
#contando quantos valores tem cada um dos itens em tipo_propriedade - casa, apartamento e lojas
df2['tipo_propriedade'].value_counts()

apartamento    273577
casa           135213
loja             8894
Name: tipo_propriedade, dtype: int64

In [73]:
#quantidade de quartos e salas (no caso das lojas)
df2['quartos'].value_counts()

1.0     144646
2.0      91697
3.0      41188
4.0      13010
5.0       2448
6.0        689
7.0        269
8.0        174
10.0       106
9.0         86
12.0        64
11.0        52
16.0        23
15.0        16
14.0        13
20.0        12
22.0        12
13.0        10
17.0         8
18.0         7
21.0         7
25.0         5
24.0         4
23.0         4
19.0         1
28.0         1
30.0         1
31.0         1
34.0         1
40.0         1
Name: quartos, dtype: int64

In [ ]:
#chamar a coluna de quartos para descobrir quais os valores contidos
sorted(pd.unique(df2['quartos']))

In [75]:
#devido a coluna quartos ser um float, forçamos ele a se tornar um numero inteiro e o NaN se tornar 0
df['quartos'] = df2['quartos'].fillna(0.0).astype(int)

In [ ]:
df2.head(10)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df2.groupby(['tipo_propriedade'])\
.tipo_propriedade\
.count()\
.sort_values()\
.plot(kind='pie', figsize=(15,5),label='Tipos de Propriedades')

In [ ]:
# Gráfico que demonstra o número de ocorrências de imóveis por cada estado, onde
# pode-se notar os estados com maior índice de imóveis divulgados
df2.groupby(['estado'])\
.estado\
.count()\
.sort_values()\
.plot(
    kind='bar',
    color='orange',
    figsize=(15,6),
    title='OCORRÊNCIAS POR ESTADO',
    xlabel='Estados',
    ylabel='Ocorrêcias'
)

In [80]:
df2.to_csv('ProjetoFinalPandas02')